# Imports and setup

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "fav-kky/gpt2-small-cs"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/947 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152M [00:00<?, ?B/s]

In [14]:
import pandas as pd

input_txt = "Dobrý den, přátelé. Dneska si"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)
        
pd.DataFrame(iterations)
     

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,"Dobrý den, přátelé. Dneska si",myslím (4.09%),můžete (2.91%),chci (2.79%),jdu (2.76%),na (2.75%)
1,"Dobrý den, přátelé. Dneska si myslím",", (95.42%)",že (3.34%),. (0.15%),", (0.14%)",to (0.07%)
2,"Dobrý den, přátelé. Dneska si myslím,",že (97.96%),že (1.23%),jak (0.06%),ze (0.06%),je (0.03%)
3,"Dobrý den, přátelé. Dneska si myslím, že",to (8.95%),je (8.16%),se (5.60%),by (4.57%),jsem (3.98%)
4,"Dobrý den, přátelé. Dneska si myslím, že to",je (15.93%),bude (13.46%),není (9.59%),bylo (7.56%),byl (4.61%)
5,"Dobrý den, přátelé. Dneska si myslím, že to je",opravdu (2.80%),docela (2.25%),pro (2.13%),dobrý (2.05%),tak (2.04%)
6,"Dobrý den, přátelé. Dneska si myslím, že to je...",velmi (3.88%),hodně (3.65%),dobrý (3.07%),tak (1.81%),něco (1.61%)
7,"Dobrý den, přátelé. Dneska si myslím, že to je...",důležité (8.04%),dobrý (6.09%),zajímavé (3.58%),dobrá (3.29%),důležitý (2.88%)


In [15]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Dobrý den, přátelé. Dneska si myslím, že to je opravdu velmi důležité


In [16]:
max_length = 128
input_txt = """V neuvěřitelném zvratu, vědci objevili, že \
stádo jednorožců žije v dříve neprobádané oblasti Tibetu. \
Ještě větší překvapení než samotný objev jednorožců bylo, \
že tito jednorožci mluví dokonale česky.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, 
                               do_sample=False)
print(tokenizer.decode(output_greedy[0]))
     

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


V neuvěřitelném zvratu, vědci objevili, že stádo jednorožců žije v dříve neprobádané oblasti Tibetu. Ještě větší překvapení než samotný objev jednorožců bylo, že tito jednorožci mluví dokonale česky.


 se v Tibetu nachází v nadmořské výšce kolem 600 metrů. V roce 1850 se v Tibetu narodil první syn a jeho otec. V roce 1850 se v Tibetu narodil první syn a jeho otec. V roce 1850 se v Tibetu narodil první syn a jeho otec. V roce 1850 se v Tibetu narodil první syn a jeho otec. V roce 1850 se v Tibetu


# Různé metody generování dalších tokenů

Greedy seacch má velkou slabinu v tom, že při výběru nejpravděpodobnějšího dalšího tokenu nedostaneme celkově koherentní větu. Lepší bude beam search, který hledá "pravděpodobnou" generovanou větu jako celek.

In [17]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

In [18]:
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [19]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

V neuvěřitelném zvratu, vědci objevili, že stádo jednorožců žije v dříve neprobádané oblasti Tibetu. Ještě větší překvapení než samotný objev jednorožců bylo, že tito jednorožci mluví dokonale česky.


 se v Tibetu nachází v nadmořské výšce kolem 600 metrů. V roce 1850 se v Tibetu narodil první syn a jeho otec. V roce 1850 se v Tibetu narodil první syn a jeho otec. V roce 1850 se v Tibetu narodil první syn a jeho otec. V roce 1850 se v Tibetu narodil první syn a jeho otec. V roce 1850 se v Tibetu

log-prob: -85.45


In [22]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=4, 
                             do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


V neuvěřitelném zvratu, vědci objevili, že stádo jednorožců žije v dříve neprobádané oblasti Tibetu. Ještě větší překvapení než samotný objev jednorožců bylo, že tito jednorožci mluví dokonale česky.


, který se nachází na okraji Tibetu, se nachází na okraji Tibetu, a to na okraji Tibetu, a to na okraji Tibetu, a to na okraji Tibetu, a to na okraji Tibetu, a to na okraji Tibetu, a to na okraji Tibetu, a to na okraji Tibetu, a to na okraji Tibetu, a to na okraji Tibetu, a

log-prob: -37.32


In [23]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


V neuvěřitelném zvratu, vědci objevili, že stádo jednorožců žije v dříve neprobádané oblasti Tibetu. Ještě větší překvapení než samotný objev jednorožců bylo, že tito jednorožci mluví dokonale česky.


, který se nachází v údolí řeky Tibet, je jedním z nejmocnějších buddhistických chrámů na světě. Je to jeden z mála buddhismů, které se dochovaly dodnes, a to nejen v Číně, ale i v dalších zemích světa.<|endoftext|>

log-prob: -81.78


In [27]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=0.8, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


V neuvěřitelném zvratu, vědci objevili, že stádo jednorožců žije v dříve neprobádané oblasti Tibetu. Ještě větší překvapení než samotný objev jednorožců bylo, že tito jednorožci mluví dokonale česky.


 se nachází v jedné z nejvýchodnějších čínských provincií obklopujících se v provincii Karmen. Je to dokonce jeden z mála čínských ostrůvků, kde celá provincie žijí. Na tuto informaci vědci upozornili především proto, že v asijském světě není tolik prostoru pro chov vody, kolik by se mohlo na toto místo určovat, od původního zdroje vody, který tady v minulosti vznikl. Tykadla


In [28]:
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


V neuvěřitelném zvratu, vědci objevili, že stádo jednorožců žije v dříve neprobádané oblasti Tibetu. Ještě větší překvapení než samotný objev jednorožců bylo, že tito jednorožci mluví dokonale česky.


 z Číny se stal v čínském prostředí prvním člověkem, který se objevil na této planetě. Ten v Tibetu viděl už přes osmdesát pět lidských druhů. Je to vlastně první člověkem v čínském pralese. Tento objev se odehrál před 100 lety, kdy na čínské zdi byla vybudována nová kolonie. To znamená, že tu vyrostlo na téměř tisíc kilometrů čtverečních této oblasti, jejíž plocha měla být až 200 tisíc
